In [ ]:
def preprocess(serialized_image):
    feature_descriptions = {
        'image' : tf.io.FixedLenFeature([], tf.string, default_value=b''),
        'label' : tf.io.FixedLenFeature([], tf.int64, default_value=0)
    }
    example = tf.io.parse_single_example(serialized_image,
                                         feature_descriptions)
    image = tf.io.parse_tensor(example['image'], out_type=tf.uint8)
    image = tf.reshape(image, shape=(28, 28, 1))
    return image, example['label']

def get_data(filepaths, shuffle_buffer_size=None, batch_size=32):
    list_files = tf.data.Dataset.list_files(filepaths)
    dataset = tf.data.TFRecordDataset(list_files, num_parallel_reads=AUTOTUNE)
    dataset = dataset.map(preprocess, num_parallel_calls=AUTOTUNE).cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    
    return dataset.batch(batch_size, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)